# チュートリアル: WikipediaのHTMLページから歴代アメリカ大統領の出生地を抽出する
DeepL翻訳

## postgresql準備
すでにpostgresqlにデータベースpob_presidentsが存在する場合は、1行目のコメントを外してドロップし、再作成してください。

In [ ]:
#! dropdb -h fonduer-postgres --if-exists pob_presidents #Uncomment to remove databases created in previous runs
! createdb -h fonduer-postgres pob_presidents

## Introduction

Webページ、ビジネスレポート、製品仕様書、科学文献に見られるような、テキスト、構造・表形式、視覚表現の組み合わせによって情報が伝達される[**リッチフォーマット**データ](https://hazyresearch.github.io/snorkel/blog/fonduer.html)から、`Fonduer`を使って関係を抽出する過程を説明します。

このチュートリアルでは、`Fonduer`を使って、大統領と副大統領の出生地をwikipediaの記事から同定します。HTMLファイルはWikipediaのマークアップ言語Wikitextから[wtf_wikipedia](https://github.com/spencermountain/wtf_wikipedia)を経由して生成されています。

チュートリアルはいくつかのパートに分かれており、それぞれが[論文](https://arxiv.org/abs/1703.05028)で概説された`Fonduer`パイプラインのフェーズと、KBCプロセスをカバーしています。

1. KBCの初期化
2. 候補者生成とマルチモーダルフィーチャー化
3. 確率的関係性分類
4. エラー解析とインタラクティブKBC

さらに、ユーザがラベリング関数を繰り返し改善することで、関係抽出の品質を向上させる方法を紹介します。

## Ph1：KBCの初期化

`Fonduer` のパイプラインの最初のフェーズでは、`Fonduer` はユーザが指定した`schema`を用いて、出力される KB が格納されるリレーショナルデータベースを初期化します。さらに、`Fonduer`は入力された`corpus`を繰り返し処理し、各文書を統一的なデータモデルに変換します。このモデルはリッチフォーマットなデータの多様性とマルチモーダル性を捉えることができます。この統一的なデータモデルは、残りのフェーズで使われる中間表現として機能します。

この前処理されたデータは、データベースに保存されます。データベースへの接続文字列は `Meta` オブジェクトに渡され、並列実行のために`PostgreSQL`データベースを初期化します。

便宜上、データベースを呼び出すための変数と、`Fonduer`パイプラインをどのレベルの並列化で実行するかを定義するための変数をいくつか初期化します。

続行する前に、`PostgreSQL`がインストールされていることと、`pob_presidents`という名前の新しいデータベースが作成されていることを確認してください。


In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline
import os
import sys
import logging

PARALLEL = 4 # assuming a quad-core machine
ATTRIBUTE = "pob_presidents"
conn_string = 'postgresql://fonduer-postgres:5432/' + ATTRIBUTE

### 1.1 入力文書の解析と統一データモデルへの変換
このオブジェクトはデータベースとの接続を自動的に管理し、中間結果を保存することができます。

In [ ]:
from fonduer import Meta, init_logging

# Configure logging for Fonduer
init_logging(log_dir="logs")

session = Meta.init(conn_string).Session()

次に、wikpediaの記事コーパスを読み込み、統一的なデータモデルに変換します。各記事はHTMLファイルで表現されます。このHTMLファイルを解析し、テキスト、構造、表形式のモダリティ情報を持つロバストな統一データモデルを作成します。各文書は互いに独立しているので、並行して解析することができることに注意してください。この機能については`PostgreSQL`に依存しています。

#### HTMLDocPreprocessorの設定
まず、ドキュメントが格納されるパスを設定し、指定したパスにあるドキュメントを読み込むための `HTMLDocPreprocessor` を定義します。`max_docs` にはパースするドキュメントの最大数を指定します。

**注意: 次のステップを実行する前に `download_data.sh` を実行しておく必要があります。

In [ ]:
from fonduer.parser.preprocessors import HTMLDocPreprocessor
from fonduer.parser import Parser

docs_path = "data/presidents/"
doc_preprocessor = HTMLDocPreprocessor(docs_path)

#### パーサーの設定
次に、PDF文書用のコーパスパーサーである`Parser`を設定します。
前処理として[spaCy](https://spacy.io/)を用いて、文書を文とトークンに分割します。
また、各文書の統一データモデルに含めるモダリティ情報を指定することが可能です。
以下では、構造情報だけでなく、[spaCy]を用いてこれらの文の品詞タグや依存構文構造などのアノテーションを提供する言語情報も有効にしています。
なお、構文解析の完了を示す進捗バーが表示された後、すべてのオブジェクトがデータベースに挿入されるまで、さらに時間がかかることに注意してください。

In [ ]:
corpus_parser = Parser(session, structural=True, lingual=True)
%time corpus_parser.apply(doc_preprocessor, parallelism=PARALLEL)

そして、簡単なデータベースクエリ（`Fonduer`が使っている[SQLAlchemy](http://www.sqlalchemy.org/)の構文で書かれています）を使って、いくつの文書や文が解析されたか、あるいはそれぞれの文書にいくつの文が含まれているかをチェックすることができます。

In [ ]:
from fonduer.parser.models import Document, Sentence

print(f"Documents: {session.query(Document).count()}")
print(f"Sentences: {session.query(Sentence).count()}")

### 1.2 コーパスをテストとトレーニングに分割する
ドキュメントを80:10:10の割合でtrain:dev:testに分割することにします。
ここでは、チュートリアルの一貫性を逆手に取り、非ランダムな順序でこれを行い、分割をそれぞれ[0]:[1]:[2]の番号で参照することに注意してください。

In [ ]:
docs = session.query(Document).order_by(Document.name).all()
ld = len(docs)

train_docs = set()
dev_docs = set()
test_docs = set()
splits = (0.7, 0.85)
data = [(doc.name, doc) for doc in docs]
data.sort(key=lambda x: x[0])
for i, (doc_name, doc) in enumerate(data):
    if i < splits[0] * ld:
        train_docs.add(doc)
    elif i < splits[1] * ld:
        dev_docs.add(doc)
    else:
        test_docs.add(doc)
from pprint import pprint

pprint([x.name for x in train_docs])

## Ph2：メンション抽出とマルチモーダル特徴量によるcandidate抽出

フェーズ1より統一的なデータモデルが与えられると、`Fonduer`はユーザーから提供された`matchers`と`throttlers`に基づく`candidate`を抽出します。
次に`Fonduer`は統一されたデータから得られるマルチモダリティな情報を活用して、各候補のマルチモーダルな特徴を提供します。

### 2.1 ミッションの抽出

まず、コーパスからメンションを抽出します。
メンションとは、`candidate`を構成するオブジェクトの種類です。
例えば、(副)大統領の名前とそれに対応する出生地のペアを抽出したい場合、名前が1つのメンションとなり、出生地がもう1つのメンションとなります。
これらのメンションを組み合わせて`candidate`を作成し、関連する文書においてどの`candidate`が正しいかを予測します。

まず、2つの`mention`の定義と命名を始めます。

In [ ]:
from fonduer.candidates.models import mention_subclass

Presidentname = mention_subclass("Presidentname")
Placeofbirth = mention_subclass("Placeofbirth")

次に、コーパス中のどのテキストスパンが各エンティティのインスタンスであるかを定義するために、`matcher`を記述します。
`matcher`は正規表現、辞書、ユーザー定義関数など様々な情報を利用することができます。
異なる技術を組み合わせることで、より質の高い`matcher`を作成することができます。
一般に、`matcher`は完全な再現性を維持しながら、可能な限り正確であることを追求する必要があります。
`matcher`に関する詳しいドキュメントは、[Read the Docs](https://fonduer.readthedocs.io/en/stable/user/candidates.html#matchers)に掲載されています。

この例では、名前に対応する文字列を定義する`matcher`を記述する必要があります。
大統領を定義する`matcher`と、出生地を定義する`matcher`になります。

#### 名前のmatcher作成

名前の`matcher`は、htmlファイル名に大統領の名前が表示されていることを利用します。
大統領についてのメンションを見つけるために、「テキストの`span`をファイル名と比較する関数」を定義します。
すでに文書を統一的なデータモデルに変換したので、テキストの各`span`はソースの文書名を参照することができ、ソースファイル名との比較が容易です。

In [ ]:
def mention_span_matches_file_name(mention):
    president_name_string = mention.get_span()
    file_name = mention.sentence.document.name.replace("_", " ")
    if president_name_string == file_name:
        return True
    else:
        return False

この関数を `LambdaFunctionMatcher` を使って、 `Fonduer` に言及する`matcher`にラップします。

In [ ]:
from fonduer.candidates.matchers import LambdaFunctionMatcher, Intersect, Union

president_name_matcher = LambdaFunctionMatcher(func=mention_span_matches_file_name)

#### 出生地のmatcher作成

出生地も正規性表現による指定が可能ですが、ここでは文書内の位置に基づいてマッチングを行います。
htmlファイルを確認すると、各ファイルにはそれぞれの大統領に関する最も重要な情報が記載された表が記載されています。
通常、表には左の列にはラベル、右の列にはそれに対応する値が存在しています。
この表情報を取得するために、「`span`が表の中にあり、同じ行の隣接するセルに「出生地」のラベルがあるかどうか」をチェックする関数を定義します。
さらに、出生地は通常、細かい位置とそれに続くそれぞれの状態に分割されていることがわかります。
ここでは、より細かい情報を収集したいので、セル内のテキストの最初の `span` だけを抽出するように指定します。

In [ ]:
from fonduer.utils.data_model_utils import get_row_ngrams


def is_in_birthplace_table_row(mention):
    if not mention.sentence.is_tabular():
        return False
    ngrams = get_row_ngrams(mention, lower=True)
    birth_place_words = set(["birth", "place"])
    if birth_place_words <= set(ngrams):
        return True
    else:
        return False


def birthplace_left_aligned_to_punctuation(mention):
    # Return false, if the cell containing the text candidate does not have any reference
    # to `sentence` objects
    for sentence in mention.sentence.cell.sentences:
        sentence_parts = sentence.text.split(",")
        for sentence_part in sentence_parts:
            if sentence_part.startswith(mention.get_span()):
                return True
    return False


# We only want one granularity of the birth place
def no_commas_in_birth_place(mention):
    if "," in mention.get_span():
        return False
    else:
        return True

出生地のプロパティを取得するためのすべての関数を定義した後、`Intersect matcher`を介してそれらを結合します。
この`matcher`は、3つの関数がすべて一致した場合にのみ`span`を選択します。

In [ ]:
birth_place_in_labeled_row_matcher = LambdaFunctionMatcher(
    func=is_in_birthplace_table_row
)
birth_place_in_labeled_row_matcher.longest_match_only = False
birth_place_no_commas_matcher = LambdaFunctionMatcher(func=no_commas_in_birth_place)
birth_place_left_aligned_matcher = LambdaFunctionMatcher(
    func=birthplace_left_aligned_to_punctuation
)

place_of_birth_matcher = Intersect(
    birth_place_in_labeled_row_matcher,
    birth_place_no_commas_matcher,
    birth_place_left_aligned_matcher,
)

この2つの`matcher`は、関係スキーマの各エンティティを定義します。

#### メンション空間(MentionSpace)の定義

次に、ドキュメントから考えられるすべてのメンションの「空間」を定義するために、抽出したい関係の各要素に対して`MentionSpace`を定義していきます。
`Fonduer`はデフォルトの`MentionSpace`を用意していますが、必要に応じてデフォルトの`MentionSpace`を拡張することも可能です。

名前の場合、`MentionSpace`は比較的単純で、それぞれの名前には少なくとも2つの単語（名、姓）が含まれまています。
ミドルネームの追加を考慮すると、1つの名前に最大で4つの単語が含まれることが予想されます。

同様に、出生地は1語から3語の`span`であることが予想されます。

これらのプロパティを定義するために、`fonduer`が提供するデフォルトの`Ngrams`クラスを使用します。

In [ ]:
from fonduer.candidates import MentionNgrams

presname_ngrams = MentionNgrams(n_max=4, n_min=2)
placeofbirth_ngrams = MentionNgrams(n_max=3)

#### メンション抽出の実行 

次に、上記で定義した`MentionSpace`と`matcher`を元に、全ての文書からメンションを抽出する`MentionExtractor`を作成します。

`MentionExtractor`のAPIは[ReadTheDocs](https://fonduer.readthedocs.io/en/stable/user/candidates.html#fonduer.candidates.MentionExtractor) で確認することができます。

In [ ]:
from fonduer.candidates import MentionExtractor

mention_extractor = MentionExtractor(
    session,
    [Presidentname, Placeofbirth],
    [presname_ngrams, placeofbirth_ngrams],
    [president_name_matcher, place_of_birth_matcher],
)

すべての文書に対して`extractor`を実行します。

In [ ]:
from fonduer.candidates.models import Mention

mention_extractor.apply(docs, parallelism=PARALLEL)
num_names = session.query(Presidentname).count()
num_pobs = session.query(Placeofbirth).count()
print(
    f"Total Mentions: {session.query(Mention).count()} ({num_names} names, {num_pobs} places of birth)"
)

### 2.2 candidateの抽出

`candidate`を構成するためのメンションの定義と抽出ができたので、次は`candidate`の抽出を行います。
メンションと同様に、まず各候補のスキーマを定義します。
この例では、上記で定義した`Presidentname`と`Placeofbirth`のメンションからなる候補を作成します。
この`candidate`を`PresidentnamePlaceofbirth`と名付けます。

In [ ]:
from fonduer.candidates.models import candidate_subclass

PresidentnamePlaceofbirth = candidate_subclass(
    "PresidentnamePlaceofbirth", [Presidentname, Placeofbirth]
)

#### Throttlersの定義

次に、余分な`candidate`を減らし、無効な`candidate`を不必要に対象にしないためのThrottlersを定義します。
`Throttler`は、`matcher`のようにフィルターとして機能し、取りこぼしはしない程度のフィルタとして作成するべきです。

ここでは、同じ表中にある`candidate`を破棄する`Throttler`を作成します。
(ただし、現在この`Throttler`は使っていません。`wtf_wikipedia`の問題で、大統領名がテーブルに存在しないことがあるからです。)

そのため、`Throttler`はスキップしています。
他の例でも`Throttler`をスキップすることは多いです。

#### CandidateExtractorの実行

これで、`candidate`抽出に必要なコンポーネントはすべて揃いました。
各`candidate`を構成する`mentions`と、余分な候補を取り除くための`throttler`を定義しました。
ここでは`CandidateExtractor`を定義して、使用する`candidate`のサブクラスと対応する`throttler`を定義します。

`CandidateExtractor`のAPIは [ReadTheDocs](https://fonduer.readthedocs.io/en/stable/user/candidates.html#fonduer.candidates.CandidateExtractor)で確認することができます。

In [ ]:
from fonduer.candidates import CandidateExtractor

candidate_extractor = CandidateExtractor(session, [PresidentnamePlaceofbirth])

ここでは、`split=0`を指定して、これらの`candidates`がトレーニングセットに属することを指定しています。
ここで、train:dev:testを[0]:[1]:[2]で分割して参照していることを思い出してください。

In [ ]:
for i, docs in enumerate([train_docs, dev_docs, test_docs]):
    candidate_extractor.apply(docs, split=i, parallelism=PARALLEL)
    print(
        f"Number of Candidates in split={i}: {session.query(PresidentnamePlaceofbirth).filter(PresidentnamePlaceofbirth.split == i).count()}"
    )

train_cands = candidate_extractor.get_candidates(split=0)
dev_cands = candidate_extractor.get_candidates(split=1)
test_cands = candidate_extractor.get_candidates(split=2)

### 2.2 マルチモーダルな特徴づけ

構造化されていないテキストを扱うのとは異なり、`Fonduer`はリッチにフォーマットされたデータを扱うため、各`candidate`をマルチモーダル特徴の基本ライブラリで特徴量化します。

#### Postgres Featurizerによる特徴量化

次に、train:dev:test(学習セット、開発セット、テストセット)の`candidate`に特徴量のアノテーションを施します。
`Fonduer` が提供する`Featurizer`を使うと、この作業を並行して行うことができ、パフォーマンスを向上させることができます。

`Featurizer` が提供する API は [ReadTheDocs](https://fonduer.readthedocs.io/en/stable/user/features.html#fonduer.features.Featurizer) で確認することができます。

In [ ]:
from fonduer.features import Featurizer

featurizer = Featurizer(session, [PresidentnamePlaceofbirth])
%time featurizer.apply(split=0, train=True, parallelism=PARALLEL)
%time F_train = featurizer.get_feature_matrices(train_cands)

In [ ]:
print(F_train[0].shape)
%time featurizer.apply(split=1, parallelism=PARALLEL)
%time F_dev = featurizer.get_feature_matrices(dev_cands)
print(F_dev[0].shape)

In [ ]:
%time featurizer.apply(split=2, parallelism=PARALLEL)
%time F_test = featurizer.get_feature_matrices(test_cands)
print(F_test[0].shape)

このフェーズでは、`candidate`の集合と特徴行列を生成します。
なお、フェーズ1と2は比較的静的な処理であり、通常、KBCの処理中に一度だけ実行されるものです。

## Ph3: 確率的関係性分類
このフェーズでは、ユーザが定義した「ラベリング関数」を適用します。
このラベリング関数には、データにラベルを付けるための様々なヒューリスティックなパターン、および [weak supervision](http://hazyresearch.github.io/snorkel/blog/weak_supervision.html)が含まれており、各`candidate`に対して、データプログラミングエンジンが使用するラベル行列を作成します。

一般に、手作業でラベル付けされた学習データはまれであり、高価になります。
ここでは、大量のラベルなし学習データを対象に、それらをラベリング関数記述でラベル付けしていきます。
ラベル付けする知識の例として、「「New」「County」「City」といった単語が含まれると都市名である」といったような知識をラベリング関数として記述します。
ラベリング関数は完全ではなく、中には非常に低品質なものが記述される場合もあるため、生成モデルを用いてその精度をモデル化していくことになります。

そして、データプログラミングを用いて機械学習モデルを学習し、`candidate`を分類する際にどの特徴が最も重要であるかを学習することができます。

### ゴールドデータ(正解データ)の読み込み
エラー解析や評価を実施するため、このチュートリアルで使用するdevセットとtestセットには既にアノテーションを施してあります。
外部で定義されたヘルパー関数を使って読み込むことにします。

In [ ]:
from fonduer.supervision.models import GoldLabel
from wiki_table_utils import gold

from fonduer.supervision import Labeler

docs = corpus_parser.get_documents()
labeler = Labeler(session, [PresidentnamePlaceofbirth])
%time labeler.apply(docs=docs, lfs=[[gold]], table=GoldLabel, train=True, parallelism=PARALLEL)

### ラベリング関数の作成

`Fonduer`では、学習する抽出モデルに学習シグナルを提供するための主要なインターフェースは、ラベリング関数(LF)を書くことです。
これは、大量の学習セットを手でラベリングするのとは対照的な作業です。

ラベリング関数は特別なものではありません。
`candidate`を入力引数として受け取り、`candidate`を「真/true」とすべきと判断した場合は「1」、「偽/False」とすべきと判断した場合は「0」、「棄権/Abstain」した場合は「-1」を返すPythonの関数に過ぎないのです。
実際には、多くのLFは単極性であり、TRUEかABSTAINを返すLF、FALSEかABSTAINを返すLFがほとんどです。
TRUEかFALSEを返すLF、もしくはTRUEかFALSEかABSTAINを返すLFは、別のLFとして分けた方がよいです。

ここでの目標は、最終的に`candidate`のうちどれが大統領名と出生地の関係を示すTRUEの`mention`であるかを予測する高性能な分類モデルを訓練することです。そのためには、低品質なラベリング関数を書けばわかりやすいです。

`Fonduer`を使えば、ラベリング関数を対象コーパスを調査して発見した直感的なパターンを使って書くことができます。
ラベリング関数を書くために使えるデータモデルユーティリティのライブラリは [Read the Docs](http://fonduer.readthedocs.io/en/stable/user/data_model_utils.html) に概要がまとめられています。

以下、このような単純なパターンのいくつかをラベリング関数の集合として表現していきます。

In [ ]:
from snorkel.labeling import labeling_function
from fonduer.utils.data_model_utils import *
import re

ABSTAIN = -1
FALSE = 0
TRUE = 1


@labeling_function()
def LF_place_of_birth_has_link(c):
    place = c.placeofbirth
    ancestor_tag_names = get_ancestor_tag_names(place)
    if len(ancestor_tag_names) > 1 and "a" in ancestor_tag_names:
        return ABSTAIN
    else:
        return FALSE


@labeling_function()
def LF_place_of_birth_is_longest_ordered_span_before_comma(c):
    place = c.placeofbirth
    place_string = place.context.get_span()
    place_sentence_string = place.context.sentence.text
    left_aligned_first_span = place_sentence_string.split(",")[0]
    if place_string == left_aligned_first_span:
        return TRUE
    else:
        return FALSE


@labeling_function()
def LF_place_in_first_sentence_of_cell(c):
    place = c.placeofbirth
    place_sentence = place.context.sentence
    place_cell = place_sentence.cell
    if place_sentence == place_cell.sentences[0]:
        return TRUE
    else:
        return FALSE


@labeling_function()
def LF_place_is_full_sentence(c):
    place = c.placeofbirth
    place_sentence = place.context.sentence
    if place.context.get_span() == place_sentence.text:
        return ABSTAIN
    else:
        return FALSE


@labeling_function()
def LF_place_not_a_US_state(c):
    place = c.placeofbirth
    place_string = place.context.get_span().lower()
    if place_string is None:
        return FALSE
    state_dictionary = set(
        x.lower()
        for x in [
            "Alabama",
            "Alaska",
            "Arizona",
            "Arkansas",
            "California",
            "Colorado",
            "Connecticut",
            "Delaware",
            "Florida",
            "Georgia",
            "Hawaii",
            "Idaho",
            "Illinois",
            "Indiana",
            "Iowa",
            "Kansas",
            "Kentucky",
            "Louisiana",
            "Maine",
            "Maryland",
            "Massachusetts",
            "Michigan",
            "Minnesota",
            "Mississippi",
            "Missouri",
            "Montana",
            "Nebraska",
            "Nevada",
            "New Hampshire",
            "New Jersey",
            "New Mexico",
            "New York",
            "North Carolina",
            "North Dakota",
            "Ohio",
            "Oklahoma",
            "Oregon",
            "Pennsylvania",
            "Rhode Island",
            "South Carolina",
            "South Dakota",
            "Tennessee",
            "Texas",
            "Utah",
            "Vermont",
            "Virginia",
            "Washington",
            "West Virginia",
            "Wisconsin",
            "Wyoming",
        ]
    )
    if place_string == "new york city":  # exception
        return TRUE
    if place_string in state_dictionary:
        return FALSE
    elif any(x in place_string for x in state_dictionary):
        return FALSE
    else:
        return ABSTAIN

次に、使いたいラベリング関数を全て一つのリストに集め、それを `Labeler` の入力として提供していきます。

In [ ]:
president_name_pob_lfs = [
    LF_place_of_birth_has_link,
    LF_place_of_birth_is_longest_ordered_span_before_comma,
    LF_place_not_a_US_state,
    LF_place_in_first_sentence_of_cell,
    LF_place_is_full_sentence,
]

### ラベリング関数の適用

すべての学習用`candidate`に対して実際にLFを実行し、`Labels`と`LabelKeys`のセット(LFの名前のみ)をデータベースに生成する必要があります。
このとき、既存の`Labels`と`LabelKeys`はすべて削除されることに注意してください。

[ReadTheDocs](https://fonduer.readthedocs.io/en/stable/user/supervision.html#fonduer.supervision.Labeler)で`Labeler`が提供するAPIを見ることができます。

In [ ]:
%time labeler.apply(split=0, lfs=[president_name_pob_lfs], train=True, parallelism=PARALLEL)
%time L_train = labeler.get_label_matrices(train_cands)

得られたラベルマトリックスに関する統計も見ることができます。
`Coverage`は、ラベリング関数が非ゼロのラベルを発行した`candidate`の割合です。
`Overlap`は、ラベリング関数が0以外のラベルを発行した`candidate`と、他のラベリング関数が0以外のラベルを発行した`candidate`の割合です。
`Conflict`は、ラベリング関数が非ゼロのラベルを発行し、別のラベリング関数が非ゼロのラベルと衝突する`candidate`の割合です。

また、すでにゴールドラベルを読み込んでいるので、[Snorkel](https://github.com/snorkel-team/snorkel)の`LFAnalysis`モジュールを使って、これらのラベリング関数とゴールドラベルを比較し、その経験的な精度を見ることができます。

In [ ]:
L_gold_train = labeler.get_gold_labels(train_cands, annotator="gold")

In [ ]:
from snorkel.labeling import LFAnalysis

LFAnalysis(
    L_train[0],
    lfs=sorted(president_name_pob_lfs, key=lambda lf: lf.name)
).lf_summary(Y=L_gold_train[0].reshape(-1))

### 生成モデルの適用

次に、LFのモデルを学習し、その精度を推定します。
モデルの学習が完了したら、LFの出力を組み合わせて、ノイズを考慮した単一の学習ラベルセットを作成し、抽出器に使用することができます。
直感的には、LFがどのように重なり合い、どのように衝突するかを観察することで、LFをモデル化することになります。
そのために、[Snorkel](https://github.com/snorkel-team/snorkel)のシングルタスクラベルモデルを利用します。

In [ ]:
from snorkel.labeling.model import LabelModel

gen_model = LabelModel(cardinality=2)
%time gen_model.fit(L_train[0], n_epochs=500, log_freq=100)

ここで、学習`candidate`に生成モデルを適用し、ノイズを考慮した学習ラベルセットを取得します。
これを学習用マージナルと呼ぶことにします。

In [ ]:
train_marginals = gen_model.predict_proba(L_train[0])

トレーニングマージンの分布を見てみます。

In [ ]:
import matplotlib.pyplot as plt

plt.hist(train_marginals[:, TRUE], bins=20)
plt.show()

### ラベリング関数を反復するためのモデル使用

生成モデルの学習が終わったので、ここで一旦停止し、ラベリング関数セットのデバッグや改良を行うことができます。
まず、LFを開発セットに適用していきます。

In [ ]:
labeler.apply(split=1, lfs=[president_name_pob_lfs], parallelism=PARALLEL)
%time L_dev = labeler.get_label_matrices(dev_cands)

In [ ]:
L_dev[0].shape

### 生成モデルの性能の解釈

この時点で、開発セットのF1スコアは約0.6～0.7となり、かなり良い値になっています。
しかし、この解釈には十分な注意が必要です。
この開発セットを参考にラベリング関数を開発し、そのラベリング関数で生成モデルを構成しているので、ここでは非常に良いスコアが期待できるのです。

実際には、この集合に対して多少過剰に適合していると思われます。
しかし、これは問題ありません。
次回は、開発セットを超えて汎化する、より強力な末端抽出モデルを学習し、ブラインドテストセット(開発中に一度も見ていないセット)で評価していきます。

### 識別モデルのトレーニング

最後に生成したノイズの多い学習ラベルを使って、抽出モデルを学習します。
このチュートリアルでは、シンプルでありながら、かなり効果的なロジスティック回帰モデルを学習します。

各`candidate`をtrueまたはfalseのメンションとして分類する識別モデルを訓練するために、訓練マージンを使用します。

Fonduerでは、新しい機械学習フレームワーク[Emmental](https://github.com/SenWu/emmental)を使用して、すべてのモデル学習をサポートします。

In [ ]:
import emmental

# Setup training config
config = {
    "meta_config": {"verbose": True},
    "model_config": {"model_path": None, "device": 0, "dataparallel": False},
    "learner_config": {
        "n_epochs": 30,
        "optimizer_config": {"lr": 0.001, "l2": 0.0},
        "task_scheduler": "round_robin",
    },
    "logging_config": {
        "evaluation_freq": 1,
        "counter_unit": "epoch",
        "checkpointing": False,
        "checkpointer_config": {
            "checkpoint_metric": {f"{ATTRIBUTE}/{ATTRIBUTE}/train/loss": "min"},
            "checkpoint_freq": 1,
            "checkpoint_runway": 2,
            "clear_intermediate_checkpoints": True,
            "clear_all_checkpoints": True,
        },
    },
}

emmental.init(Meta.log_path)
emmental.Meta.update_config(config=config)

In [ ]:
# Collect word counter from training data
from fonduer.learning.utils import collect_word_counter

word_counter = collect_word_counter(train_cands)

In [ ]:
# Generate word embedding module for LSTM model
# (in Logistic Regression, we generate it since Fonduer dataset requires word2id dict)
from emmental.modules.embedding_module import EmbeddingModule

arity = 2

# Geneate special tokens
specials = []
for i in range(arity):
    specials += [f"~~[[{i}", f"{i}]]~~"]

emb_layer = EmbeddingModule(
    word_counter=word_counter, word_dim=300, specials=specials
)

In [ ]:
# Generate dataloader for training set
from emmental.data import EmmentalDataLoader
from fonduer.learning.dataset import FonduerDataset
import numpy as np

# Filter out noise samples
diffs = train_marginals.max(axis=1) - train_marginals.min(axis=1)
train_idxs = np.where(diffs > 1e-6)[0]

train_dataloader = EmmentalDataLoader(
    task_to_label_dict={ATTRIBUTE: "labels"},
    dataset=FonduerDataset(
        ATTRIBUTE,
        train_cands[0],
        F_train[0],
        emb_layer.word2id,
        train_marginals,
        train_idxs,
    ),
    split="train",
    batch_size=100,
    shuffle=True,
)

In [ ]:
# Creaet task and model, and perform training
from emmental.model import EmmentalModel
from fonduer.learning.task import create_task
from emmental.learner import EmmentalLearner

tasks = create_task(
    ATTRIBUTE, 2, F_train[0].shape[1], 2, emb_layer, model="LogisticRegression"
)

model = EmmentalModel(name=f"{ATTRIBUTE}_task")

for task in tasks:
    model.add_task(task)

emmental_learner = EmmentalLearner()
emmental_learner.learn(model, [train_dataloader])

## テストセットでの評価
この最後のセクションでは、ブラインドテストセットにおける抽出モデルのパフォーマンスを得ます(split 2)。
まず、先ほどのテストセットのラベルとゴールド`candidate`を読み込みます。

In [ ]:
import numpy as np
from wiki_table_utils import entity_level_f1

今度は、判別モデルを使ったスコアリングを行います。

In [ ]:
# Generate dataloader for test data
test_dataloader = EmmentalDataLoader(
    task_to_label_dict={ATTRIBUTE: "labels"},
    dataset=FonduerDataset(
        ATTRIBUTE, test_cands[0], F_test[0], emb_layer.word2id, 2
    ),
    split="test",
    batch_size=100,
    shuffle=False,
)

In [ ]:
gold_file = "data/president_tutorial_gold.csv"

test_preds = model.predict(test_dataloader, return_preds=True)
positive = np.where(np.array(test_preds["probs"][ATTRIBUTE])[:, TRUE] > 0.6)
true_pred = [test_cands[0][_] for _ in positive[0]]
%time (TP, FP, FN) = entity_level_f1(true_pred, gold_file, test_docs)